In [1]:
import pandas as pd
import openai
import requests
import json
from tqdm import tqdm



In [2]:
# Load the CSV file with extracted grant information
input_csv_path = '/Users/miguelemb/Downloads/Projects/Grant scrapper/data/scraped_data/scraped_all_grants.csv'


In [3]:
# Limit the number of grants for testing

grants_df = pd.read_csv(input_csv_path)


In [4]:
display(grants_df)

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,Extracted Text
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
...,...,...,...,...,...,...,...,...
505,"=HYPERLINK(""https://www.grants.gov/search-resu...",2024 Bureau of Indian Education: Tribal Educat...,DOI-BIA,posted,08/13/2024,10/15/2024,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
506,"=HYPERLINK(""https://www.grants.gov/search-resu...",Climate Program Office for FY 2012,DOC,posted,07/06/2011,NaN,https://www.grants.gov/search-results-detail/1...,An official website of the United States gover...
507,"=HYPERLINK(""https://www.grants.gov/search-resu...",English-Language Program for Disadvantaged Com...,DOS-AZE,forecasted,03/18/2024,NaN,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...
508,"=HYPERLINK(""https://www.grants.gov/search-resu...",USAID/DCHA/FFP International Emergency Food As...,USAID,posted,05/06/2013,NaN,https://www.grants.gov/search-results-detail/2...,An official website of the United States gover...


In [5]:
# Set up the OpenAI API key
openai.api_key = 'sk-NBbb1i54-gKgvS54YsbXmSpm2KhPKNHUyLBCyBsoj2T3BlbkFJli2IEFcxqOTg20t8EwELIWZGDLreEzoGne9G40MwMA'

In [6]:
# Detailed description of Founder School
founder_school_description = (
    "Founder School is a nonprofit organization focused on empowering underrepresented entrepreneurs across the United States, Caribbean, Central America, and South America. "
    "We provide resources, mentorship, and funding opportunities for individuals from marginalized communities who want to start or scale their businesses. "
    "Our primary focus is on promoting inclusive innovation and economic empowerment by providing access to networks, business education, technology training, and support. "
    "Founder School helps founders develop sustainable business models, refine their skills, and secure funding, particularly in underserved regions where traditional venture capital is less accessible. "
    "We represent Black, Latino, women, and other underrepresented entrepreneurs, with a focus on technology companies, especially in crypto, AI, and related fields. "
    "Our mission is to foster a supportive community where aspiring entrepreneurs can learn, connect, and grow, ultimately transforming their ideas into successful ventures."
)


In [7]:
# Define the columns that we will be adding to the DataFrame
columns = ['Apply Decision', 'Reason for Decision', 'Funding Amount','Additional Information URL']

In [8]:
# Ensure these columns are present in the original DataFrame
for col in columns:
    if col not in grants_df.columns:
        grants_df[col] = None


In [9]:
# Initialize a counter for API usage
api_usage_count = 0

In [10]:
grants_df.head()

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,Extracted Text,Apply Decision,Reason for Decision,Funding Amount,Additional Information URL
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,None,None,None,None


In [11]:

# Function to analyze grants using GPT-4o-mini
def analyze_grant(grant_info):
    # Create a prompt based on grant information
    prompt = (
        f"Given the following grant information, determine if it is a good fit for Founder School. "
        f"Founder School is described as follows: {founder_school_description}. "
        f"Provide a summary analysis of the grant's fit for Founder School in the following format: "
        f"'Decision: [Apply/Not Apply/Maybe] - Reason: [Reason for the decision] - Funding Amount: [Amount, if available] - Additional Information URL: [URL, if available]'. "
        f"Only provide an answer if you have all the necessary information to make a confident decision. If unsure, say 'Not Sure'."
        f"\nGrant Information:\n{grant_info}"
    )
    
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json=data
    )
    response_json = response.json()
    return response_json['choices'][0]['message']['content'].strip()

In [12]:
# Loop through each grant and apply GPT analysis with a progress bar
for idx in tqdm(range(len(grants_df)), desc="Analyzing Grants"):
    grant_info = grants_df.iloc[idx].to_dict()
    gpt_analysis = analyze_grant(grant_info)
    
    # Increment API usage count
    api_usage_count += 1
    
    # Split GPT analysis into decision, reason, funding amount, and additional information URL
    if "Decision:" in gpt_analysis and "Reason:" in gpt_analysis and "Funding Amount:" in gpt_analysis and "Additional Information URL:" in gpt_analysis:
        try:
            decision_part = gpt_analysis.split("Decision:")[1].split("- Reason:")[0].strip()
            reason_part = gpt_analysis.split("Reason:")[1].split("- Funding Amount:")[0].strip()
            funding_amount_part = gpt_analysis.split("Funding Amount:")[1].split("- Additional Information URL:")[0].strip()
            additional_info_url_part = gpt_analysis.split("Additional Information URL:")[1].strip()
            grants_df.at[idx, 'Apply Decision'] = decision_part
            grants_df.at[idx, 'Reason for Decision'] = reason_part
            grants_df.at[idx, 'Funding Amount'] = funding_amount_part
            grants_df.at[idx, 'Additional Information URL'] = additional_info_url_part
        except IndexError:
            grants_df.at[idx, 'Apply Decision'] = "Error parsing GPT response"
            grants_df.at[idx, 'Reason for Decision'] = "Error parsing GPT response"
            grants_df.at[idx, 'Funding Amount'] = "Error parsing GPT response"
            grants_df.at[idx, 'Additional Information URL'] = "Error parsing GPT response"
    else:
        grants_df.at[idx, 'Apply Decision'] = "Error parsing GPT response"
        grants_df.at[idx, 'Reason for Decision'] = "Error parsing GPT response"
        grants_df.at[idx, 'Funding Amount'] = "Error parsing GPT response"
        grants_df.at[idx, 'Additional Information URL'] = "Error parsing GPT response"

# Print the total API usage
print(f"Total API calls made: {api_usage_count}")


Analyzing Grants:   0%|          | 0/510 [00:00<?, ?it/s]

Analyzing Grants: 100%|██████████| 510/510 [13:57<00:00,  1.64s/it]

Total API calls made: 510


In [13]:
# Add a column to check if all columns were read before providing an answer
grants_df['All Columns Read'] = grants_df.apply(lambda row: 'Yes' if all(pd.notna(row)) else 'No', axis=1)

In [14]:
# Save the analyzed data to a new CSV file
grants_df.to_csv("/Users/miguelemb/Downloads/Projects/Grant scrapper/data/agent_anlysis/gpt_analysis_results.csv", index=False)

In [15]:
display(grants_df)

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,Extracted Text,Apply Decision,Reason for Decision,Funding Amount,Additional Information URL,All Columns Read
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is focused on supporting U.S.-based ...,"$10,000,000 (total program funding, with indiv...",[NSF Publication 24-595](https://www.grants.go...,Yes
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is focused on STEM research infrastr...,"Estimated Total Program Funding: $31,500,000; ...",[NSF Grant Opportunity 23-588](https://www.gra...,Yes
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically aimed at small busin...,Not specified,[https://www.grants.gov/search-results-detail/...,Yes
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically targeted at small bu...,Not specified,[https://grants.nih.gov/grants/guide/pa-files/...,Yes
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is focused on supporting small busin...,Not specified,https://grants.nih.gov/grants/guide/pa-files/P...,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,"=HYPERLINK(""https://www.grants.gov/search-resu...",2024 Bureau of Indian Education: Tribal Educat...,DOI-BIA,posted,08/13/2024,10/15/2024,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant is specifically targeted towards fed...,"$2,000,000 (with awards ranging from $50,000 t...",[2024 Bureau of Indian Education: Tribal Educa...,Yes
506,"=HYPERLINK(""https://www.grants.gov/search-resu...",Climate Program Office for FY 2012,DOC,posted,07/06/2011,NaN,https://www.grants.gov/search-results-detail/1...,An official website of the United States gover...,Not Apply,The grant opportunity focuses on climate resea...,"Estimated total program funding is $15,500,000...",[NOAA-OAR-CPO-2012-2003041](https://www.grants...,No
507,"=HYPERLINK(""https://www.grants.gov/search-resu...",English-Language Program for Disadvantaged Com...,DOS-AZE,forecasted,03/18/2024,NaN,https://www.grants.gov/search-results-detail/3...,An official website of the United States gover...,Not Apply,The grant focuses specifically on English-lang...,"$100,000 total program funding, with individua...",[https://www.grants.gov/search-results-detail/...,No
508,"=HYPERLINK(""https://www.grants.gov/search-resu...",USAID/DCHA/FFP International Emergency Food As...,USAID,posted,05/06/2013,NaN,https://www.grants.gov/search-results-detail/2...,An official website of the United States gover...,Not Apply,The grant opportunity is focused on internatio...,Not specified,https://www.grants.gov/search-results-detail/2...,No
